In [1]:
import uuid
import random

from tqdm import tqdm
from threading import Thread
from threading import Thread, Lock

from cassandra.cluster import Cluster, BatchStatement
from cassandra.policies import RetryPolicy, ExponentialReconnectionPolicy

from utils.query_utils import *
from cassandra import ConsistencyLevel
from cassandra.cluster import ExecutionProfile, EXEC_PROFILE_DEFAULT

from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel
from cassandra.cluster import Cluster, ExecutionProfile, EXEC_PROFILE_DEFAULT
from cassandra.policies import WhiteListRoundRobinPolicy, DowngradingConsistencyRetryPolicy
from cassandra.query import tuple_factory
import time

In [2]:
profile = ExecutionProfile(
    retry_policy = DowngradingConsistencyRetryPolicy,
    consistency_level = ConsistencyLevel.TWO,
    serial_consistency_level = ConsistencyLevel.LOCAL_SERIAL,
    request_timeout = 200,
)

CLUSTER_IDS = ['172.21.0.2']
KEYSPACE = 'library_keyspace'

cluster = Cluster(CLUSTER_IDS, execution_profiles={EXEC_PROFILE_DEFAULT: profile})
session = cluster.connect()
session.set_keyspace(KEYSPACE)

In [3]:
res = get_all_reservations(session)
for r in res:
    reservation_id = r.reservation_id
    user_id = r.user_id
    user_name = r.user_name
    book_id = r.book_id
    book_name = r.book_name
    
reservation_id

UUID('e8228912-65fa-4332-a25a-5663b62d3501')

In [4]:
def add_reservation_fast(session, reservation_id, user_id, user_name, book_name, book_id, timeout = 120):
    insert_reservation_query = """
        INSERT INTO reservations (reservation_id, user_id, user_name, book_name, book_id)
        VALUES (%s, %s, %s, %s, %s);
    """
    try:
        book = get_book(session, book_id = book_id)
        if book is not None and not book.is_reserved:
            set_book_reserved(session, book_id=book_id, reserved = True, timeout = timeout)
            #append_user_reservation(session, user_id, user_name, reservation_id, timeout = timeout)
            session.execute(insert_reservation_query, [reservation_id, user_id, user_name, book_name, book_id], timeout=timeout)
    except InvalidRequest as e:
        print("Error occurred while inserting reservation:", e)

In [5]:
def add_reservation_slow(session, reservation_id, user_id, user_name, book_name, book_id, timeout = 120):
    insert_reservation_query = """
        INSERT INTO reservations (reservation_id, user_id, user_name, book_name, book_id)
        VALUES (%s, %s, %s, %s, %s);
    """
    try:
        book = get_book(session, book_id = book_id)
        if book is not None and not book.is_reserved:
            set_book_reserved(session, book_id=book_id, reserved = True, timeout = timeout)
            append_user_reservation(session, user_id, user_name, reservation_id, timeout = timeout)
            session.execute(insert_reservation_query, [reservation_id, user_id, user_name, book_name, book_id], timeout=timeout)
    except InvalidRequest as e:
        print("Error occurred while inserting reservation:", e)

In [6]:
class Add_Reservation_Handler:
    def __init__(self, session):
        self.insert_data = []  # Initialized as an empty list
        self.counter = 0  # Initialized as 0
        self.session = session

    def add_data(self, data):
        self.insert_data.append(data)
        self.counter += 1

    def display_data(self):
        print("Inserted Data:", self.insert_data)
        print("Counter:", self.counter)

    def batch_set_book_reserved(self, session, book_id, reserved = True, timeout = 120):
        pass

    def add_batch_reservation(self, reservation_id, user_id, user_name, book_name, book_id, batch_size = 1000, forceful_exec = False, timeout = 120):
        try:
            book = get_book(self.session, book_id = book_id)
            if book is not None and not book.is_reserved:
                self.insert_data.append((reservation_id, user_id, user_name, book_name, book_id))
                self.counter += 1
                if forceful_exec or self.counter == batch_size:
                    self.batch_set_book_reserved(book_id=book_id, reserved = True, timeout = timeout)
                    self.batch_append_user_reservation(self.session, user_id, user_name, reservation_id, timeout = timeout)
                    insert_reservation_query = self.session.prepare("""
                                                                    INSERT INTO reservations (reservation_id, user_id, user_name, book_name, book_id)
                                                                    VALUES (?, ?, ?, ?, ?);""")
                    batch_add = BatchStatement()
                    for example in self.insert_data:
                        batch_add.add(insert_reservation_query, example[0], example[1], example[2], example[3], example[4])
                    
                    self.session.execute(batch_add, timeout=timeout)
                    self.counter = 0
                    self.insert_data = []
        except InvalidRequest as e:
            print("Error occurred while inserting reservation:", e)

In [ ]:
    print("Adding books")
    book_ids = []
    book_names = []
    insert_statement_books = session.prepare("""
        INSERT INTO books (book_id, book_name, is_reserved) VALUES (?, ?, ?)
    """)
    for i in range(200):
        batch_books = BatchStatement()
        for j in range(10):
            book_id = uuid.uuid4()
            book_ids.append(book_id)
            book_name = f'Book {i*10+j+1}'
            book_names.append(book_name)
            batch_books.add(insert_statement_books, (book_id, book_name, False))
        try:
            session.execute(batch_books, timeout=120)
        except InvalidRequest as e:
            raise e

In [7]:
def add_batch_reservation(session, reservation_id, user_id, user_name, book_name, book_id, batch_size = 1000, forceful_exec = False, counter = 0):
    insert_reservation_query = """
        INSERT INTO reservations (reservation_id, user_id, user_name, book_name, book_id)
        VALUES (%s, %s, %s, %s, %s);
    """
    try:
        book = get_book(session, book_id = book_id)
        if book is not None and not book.is_reserved:

            set_book_reserved(session, book_id=book_id, reserved = True, timeout = timeout)
            append_user_reservation(session, user_id, user_name, reservation_id, timeout = timeout)
            session.execute(insert_reservation_query, [reservation_id, user_id, user_name, book_name, book_id], timeout=timeout)
    except InvalidRequest as e:
        print("Error occurred while inserting reservation:", e)    
    return counter

def cancel_batch_reservation(session, reservation_id, batch_size = 1000, forceful_exec = False, prepared = False):
    pass

In [8]:
def test_reserve_cancel_fast(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        add_reservation_fast(session, reservation_id, user_id, user_name, book_name, book_id)
        cancel_reservation(session, reservation_id)

    add_reservation_fast(session, reservation_id, user_id, user_name, book_name, book_id)

def test_reserve_cancel_slow(session, reservation_id, user_id, user_name, book_id, book_name, repeat_num=10_000):
    # Creates and cancels a reservation 10_000 times

    for _ in tqdm(range(repeat_num)):
        add_reservation_slow(session, reservation_id, user_id, user_name, book_name, book_id)
        cancel_reservation(session, reservation_id)

    add_reservation_slow(session, reservation_id, user_id, user_name, book_name, book_id)

In [9]:
start_time = time.time()
test_reserve_cancel_fast(session, reservation_id, user_id, user_name, book_id, book_name)
end_time = time.time()
print(f"Execution Time: {end_time - start_time}")

100%|██████████| 10000/10000 [00:58<00:00, 169.56it/s]

Execution Time: 58.98241949081421


In [10]:
start_time = time.time()
test_reserve_cancel_slow(session, reservation_id, user_id, user_name, book_id, book_name)
end_time = time.time()
print(f"Execution Time: {end_time - start_time}")

100%|██████████| 10000/10000 [03:18<00:00, 50.33it/s]

Execution Time: 198.7042052745819


In [11]:
start_time = time.time()
test_reserve_cancel_fast(session, reservation_id, user_id, user_name, book_id, book_name)
end_time = time.time()
print(f"Execution Time: {end_time - start_time}")

100%|██████████| 10000/10000 [02:56<00:00, 56.51it/s]

Execution Time: 176.95874738693237
